In [1]:
import numpy as np
import pandas as pd 
import copy
import os

import matplotlib.pyplot as plt
import seaborn as sns

from random import shuffle, randint

from numpy import mean, std, dstack

from pandas import read_csv

from keras import backend as K
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Model, Sequential
from keras.layers import (Input, Dense, Flatten, Dropout, Conv1D, LSTM, GRU,
                          TimeDistributed, GlobalAveragePooling1D, MaxPooling1D)
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.merge import concatenate

from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler

from keras.optimizers import Adam, RMSprop,SGD
from keras.utils import to_categorical
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

from scipy import signal

%matplotlib inline
with pd.option_context("display.max_rows", 10, "display.max_columns", 20):
    print(pd.get_option("display.max_rows")) 
    print(pd.get_option("display.max_columns"))
    
# Check GPU support 
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

Using TensorFlow backend.


10
20
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3506142393465178657
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 2524660210766233960
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 6057561278872128267
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3009871872
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14120789644656022394
physical_device_desc: "device: 0, name: GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [2]:
from os import chdir as cd
cd('..')

In [3]:
from seriem_temporis.display_functions import display_block_of_values
from seriem_temporis.controller import SignalController, KasperskySetSignalController
from neural_models.models import SplitConvolutionalAnomalyDetector

In [4]:
neural_model = SplitConvolutionalAnomalyDetector()

Number of columns: 60
NaN Values: False
Number of columns after clean: 55
Successfully scaled control_results
savgol filter smoothing successful
Input shape: (2988, 500, 55) 
Output shape: (2988, 500, 2)
Instructions for updating:
Colocations handled automatically by placer.


In [6]:
neural_model.fit_model(verbose=1, epochs_per_step=5, batch_size=128)

Number of columns: 60
NaN Values: False
Number of columns after clean: 55
Successfully scaled control_results
savgol filter smoothing successful
Input shape: (2988, 500, 55) 
Output shape: (2988, 500, 2)
Instructions for updating:
Use tf.cast instead.
Epoch 1/5
2988/2988 [==============================] - 12s 4ms/step - loss: 0.0296 - accuracy: 0.9999
Epoch 2/5
2988/2988 [==============================] - 9s 3ms/step - loss: 1.7076e-04 - accuracy: 0.9999
Epoch 3/5
2988/2988 [==============================] - 9s 3ms/step - loss: 1.4215e-04 - accuracy: 0.9999
Epoch 4/5
2988/2988 [==============================] - 9s 3ms/step - loss: 1.3613e-04 - accuracy: 0.9999
Epoch 5/5
2988/2988 [==============================] - 9s 3ms/step - loss: 1.3378e-04 - accuracy: 0.9999
Number of columns: 60
NaN Values: False
Number of columns after clean: 55
Successfully scaled control_results
savgol filter smoothing successful
Input shape: (2988, 500, 55) 
Output shape: (2988, 500, 2)
Epoch 1/5
1152/2988 [=

KeyboardInterrupt: 

Initialise neural model

### Estimate model results 

In [8]:
model = neural_model.get_model()

In [13]:
from neural_models.generators import dataset_reader
DATASET_READER = dataset_reader('/media/jurgen/Новый том/Sygnaldatasets/kaspersky/attacks/')

In [14]:
for input_train, output_train in DATASET_READER:
    y_pred = model.predict([input_train, input_train, input_train])
    y_anomaly_test = output_train[:,:,1]
    y_anomaly_prediction = y_pred[:,:,1]
    decision_threshold = 0.4
    y_anomaly_prediction = (y_anomaly_prediction > decision_threshold).astype(int)
    print(np.unique(y_anomaly_prediction, return_counts=True))
    print(np.unique(y_anomaly_test, return_counts=True))
    print(y_anomaly_prediction.shape)
    print(y_anomaly_test.shape)
    cummulative_test = y_anomaly_test.reshape(y_anomaly_test.shape[0]*y_anomaly_test.shape[1])
    cummulative_test_pred = y_anomaly_prediction.reshape(y_anomaly_test.shape[0]*y_anomaly_test.shape[1])
    _f1_score = f1_score(cummulative_test, cummulative_test_pred)
    _recall_score = recall_score(cummulative_test, cummulative_test_pred)
    _precision_score = precision_score(cummulative_test, cummulative_test_pred)
    IOU = []
    for idx, i in enumerate(y_anomaly_prediction):
        OL = 0
        UN = 0
        for _idx, result in enumerate(i):
            if result ==1 and y_anomaly_test[idx,_idx]==1:
                OL += 1  
            if result ==1 or y_anomaly_test[idx,_idx]==1:
                UN += 1
        if OL != 0 and UN != 0:
            IOU.append(OL/UN)
    print(_f1_score, _recall_score, _precision_score)
    print(mean(IOU))

Number of columns: 60
NaN Values: False
Number of columns after clean: 55
Successfully scaled control_results
savgol filter smoothing successful
Input shape: (2988, 500, 55) 
Output shape: (2988, 500, 2)
(array([0]), array([1494000]))
(array([0., 1.], dtype=float32), array([1493656,     344]))
(2988, 500)
(2988, 500)


/home/jurgen/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jurgen/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.0 0.0 0.0
nan


/home/jurgen/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/jurgen/.local/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Number of columns: 60
NaN Values: False
Number of columns after clean: 55
Successfully scaled control_results
savgol filter smoothing successful
Input shape: (2988, 500, 55) 
Output shape: (2988, 500, 2)


KeyboardInterrupt: 

In [7]:
y_pred = neural_model.predict([X_test,X_test,X_test])
y_anomaly_test = Y_test[:,:,1]
y_anomaly_prediction = y_pred[:,:,1]

NameError: name 'model_cont' is not defined

In [54]:
decision_threshold = 0.4
y_anomaly_prediction = (y_anomaly_prediction > decision_threshold).astype(int)

In [55]:
print(np.unique(y_anomaly_prediction, return_counts=True))
print(np.unique(y_anomaly_test, return_counts=True))
print(y_anomaly_prediction.shape)
print(y_anomaly_test.shape)

(array([0]), array([2934000]))
(array([0., 1.], dtype=float32), array([2933620,     380]))
(1956, 1500)
(1956, 1500)


In [42]:
cummulative_test = y_anomaly_test.reshape(y_anomaly_test.shape[0]*y_anomaly_test.shape[1])
cummulative_test_pred = y_anomaly_prediction.reshape(y_anomaly_test.shape[0]*y_anomaly_test.shape[1])

In [43]:
print(cummulative_test.shape)
print(cummulative_test_pred.shape)

(2934000,)
(2934000,)


In [44]:
_f1_score = f1_score(cummulative_test, cummulative_test_pred)
_recall_score = recall_score(cummulative_test, cummulative_test_pred)
_precision_score = precision_score(cummulative_test, cummulative_test_pred)

/home/jurgen/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [45]:
print(_f1_score, _recall_score, _precision_score)

0.0 0.0 0.0


In [49]:
IOU = []
for idx, i in enumerate(y_anomaly_prediction):
    OL = 0
    UN = 0
    for _idx, result in enumerate(i):
        if result ==1 and y_anomaly_test[idx,_idx]==1:
            OL += 1  
        if result ==1 or y_anomaly_test[idx,_idx]==1:
            UN += 1
    if OL != 0 and UN != 0:
        IOU.append(OL/UN)

In [50]:
print(mean(IOU))

nan


/home/jurgen/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/jurgen/.local/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
